In [1]:
import cv2
import mediapipe as mp
import time


WEBCAM_ID = 2

CREATE_VIDEO = '../output/face_mesh.mp4'

In [5]:
video_source = WEBCAM_ID

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

mp_face_mesh = mp.solutions.face_mesh
    
cap = cv2.VideoCapture(video_source)

video_writer = None

drawing_spec = mp_draw.DrawingSpec(thickness=1, circle_radius=1)

with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face:
    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:

        while cap.isOpened():
            success, image = cap.read()

            start = time.time()

            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            face_results = face.process(image)
            hand_results = hands.process(image)

            image.flags.writeable = True

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if face_results.multi_face_landmarks:
                for lmks in face_results.multi_face_landmarks:
                    #print(lmks)
                    mp_draw.draw_landmarks(image, lmks, 
                                           connections=mp_face_mesh.FACEMESH_CONTOURS,
                                           landmark_drawing_spec=drawing_spec,
                                           connection_drawing_spec=drawing_spec)
            
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    #print(hand_landmarks)
                    mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            end = time.time()
            total_time = end - start
            fps = 1 / total_time

            cv2.putText(image, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
            cv2.imshow('Hand Detection & Face Mesh', image)

            if CREATE_VIDEO is not False:
                if video_writer is None:
                    height,width,layers = image.shape
                    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
                    #self._out = VideoWriter(self._name, self._fourcc, 20.0, (640,480))
                    video_writer = cv2.VideoWriter(CREATE_VIDEO, fourcc, 20.0, (width,height))
                video_writer.write(image)

            if cv2.waitKey(5) & 0xFF == 27:
                break

cap.release()
cv2.destroyAllWindows()
if video_writer is not None:
    video_writer.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [3]:
cap.release()